<a name="top"></a>
# Introduction to Python Programming for Bioinformatics. Lesson 7

<details>
<summary>
About this notebook
</summary>

This notebook was originally written by [Marc Cohen](https://github.com/mco-gh), an engineer at Google. The original source can be found on [Marc's short link service](https://mco.fyi/), and starts with [Python lesson 0](https://mco.fyi/py0), and I encourage you to work through that notebook if you find some details missing here.

Rob Edwards edited the notebook, adapted it for bioinformatics, using some simple geneticy examples, condensed it into a single notebook, and rearranged some of the lessons, so if some of it does not make sense, it is Rob's fault!

It is intended as a hands-on companion to an in-person course, and if you would like Rob to teach this course (or one of the other courses) don't hesitate to get in touch with him.

</details>
<details>
<summary>
Using this notebook
</summary>


You can download the original version of this notebook from [GitHub](https://linsalrob.github.io/ComputationalGenomicsManual/Python/Python_Lesson_7.ipynb) and from [Rob's Google Drive]()

**You should make your own copy of this notebook by selecting File->Save a copy in Drive from the menu bar above, and then you can edit the code and run it as your own**

There are several lessons, and you can do them in any order. I've tried to organise them in the order I think most appropriate, but you may disagree!
</details>


<a name="lessons"></a>

# Lesson Links

* [Lesson 7 - Reading and Writing Files](#Lesson-7---Reading-and-Writing-Files)
  * [Reading a fasta file](#Reading-a-fasta-file)
  * [Writing to files](#Writing-to-files)
  * [Reading and writing gzip files](#Reading-and-writing-gzip-files)

Previous Lesson: [GitHub](Python_Lesson_06.ipynb) | [Google Colab](https://colab.research.google.com/drive/1iLE4rkhVxige0za_H0ehtb9NNzzxQIwu)

Next Lesson: [GitHub](Python_Lesson_08.ipynb) | [Google Colab](https://colab.research.google.com/drive/1ytI6exHPmHDvtzwRsS-zELWlJBOig9L9)


# Lesson 7 - Reading and Writing Files

In bioinformatics almost all our data is stored in text files. Sometimes we compress them to save space, but DNA sequences, read mapping, feature counts, RNAseq counts, and everything else are saved in files.

>**TIP**: When you are saving data, use `.tsv` - tab separated variables - files, that separate things with tabs. Often, people use commas to separate their data, but that becomes more complex when your data also contains commas (eg. in protein functions, names, and other things). Nothing contains tabs, and so they are essentially a unique separator!

Python has a built-in function called `open()` that we usually use with two parameters: the filename to open and the `mode` to open it:

* r: open the file to read
* w: open the file to write
* a: open the file to append to

_Note_: If you open a file in mode `w` that already exists, it will delete your file and overwrite it!

Almost always we use `r` or `w` to read and write from a file

## Reading a fasta file

The [fasta format](https://en.wikipedia.org/wiki/FASTA_format) is one of the simplest file formats for storing DNA sequences.

The identifier line(s) begin with `>`, while the other lines are sequence lines.

Here's a function that reads a fasta file and returns a dictionary of all the sequences in the file

In [ ]:
def read_fasta(filename):
  """
  Read a fasta file and return a dictionary of sequences.
  :param filename: The name of the file to read.
  :type filename: str
  :return: A dictionary of sequences.
  :rtype: dict
  """

  sequences = {}
  with open(filename, "r") as f:
    for line in f:
      line = line.strip()
      if line.startswith(">"):
        name = line[1:]
        sequences[name] = ""
      else:
        sequences[name] += line
  return sequences

Note the line:

```
with open(filename, "r") as f:
```

this starts a new block of code and does a few things:
- opens the file `filename` for reading (`r`)
- saves the open file as variable `f` for dealing with later
- when the `with open ...` block exits, Python automatially closes the file and cleans up any access. This is _less_ important (but still important) when reading a file, but very important when writing a file.


We can use that to read a fasta file. Download the [crassphage Bc 01 sequence](https://raw.githubusercontent.com/linsalrob/ComputationalGenomicsManual/master/Python/Bc01.fasta) to your computer and then open the file folder icon in the left bar, and drop the file there. Google Colab will warn you that the file is temporary, and that's OK, because you can always download it again!

We'll read the fasta and print the sequences and their lengths

In [ ]:
sequences = read_fasta("Bc01.fasta")
for name, sequence in sequences.items():
  print(f"{name}: {len(sequence)}")

Lets extend that and calculate the reverse complement of each sequence using the method we defined earlier.

_Note:_ If you have come back here make sure that you run the block of code with the `reverse_complement()` function above


In [ ]:
def reverse_complement(dna):
    """
    Reverse complement a DNA sequence
    :param dna: The DNA sequence
    :type dna: str
    :return: The reverse complement of the DNA sequence
    :rtype: str
    """
    complements = str.maketrans('acgtrymkbdhvACGTRYMKBDHV', 'tgcayrkmvhdbTGCAYRKMVHDB')
    rcseq = dna.translate(complements)[::-1]
    return rcseq

In [ ]:
sequences = read_fasta("Bc01.fasta")
for name, sequence in sequences.items():
  print(f">{name}\n{reverse_complement(sequence)}")

## Writing to files

To open a file for writing, we use the same notation as we did for reading the file, but instead we open it in mode `w` for writing.

We modify our print statement to include the file to write to.

In [ ]:
filename = "Bc01_rc.fasta" # the reverse complement of the sequence
with open(filename, "w") as f:
  for name, sequence in sequences.items():
    print(f">{name}\n{reverse_complement(sequence)}", file=f)

Now you have a new file called `Bc01_rc.fasta` in your files tab that contains the reverse complement of the sequence.

## Reading and writing gzip files

We _often_ compress files with a program called [gzip](https://en.wikipedia.org/wiki/Gzip) and the files have a name that ends `.gz`.

We can use this to determine whether we are going to read a text file or a gzip file, and then use the `gzip.open()` instead of the regular `open()` function.

In the next section we're going to learn about modules, but for now, we're going to use a module called `gzip` and we need to `import` it so we can access it.

`gzip.open()` is essentially a drop in replacement, but we have to add one additional parameter which tells `gzip` whether to return binary data (ie. raw data) or to convert it to text. We assign that to a variable if the file name ends `.gz`, otherwise we use the default `open()` function. Note the more compressed format of `if ... else` that is often used to put everything on one line.

Here's our modified `read_fasta()` function that accepts both plain text and gzip compressed text.

In [ ]:
def read_fasta(filename):
  """
  Read a fasta file and return a dictionary of sequences. The file can be plain text or gzip compressed.
  :param filename: The name of the file to read.
  :type filename: str
  :return: A dictionary of sequences.
  :rtype: dict
  """

  # normally we would put the import function at the beginning of the code
  import gzip

  sequences = {}
  opener = gzip.open if filename.endswith(".gz") else open
  with opener(filename, "rt") as f:
    for line in f:
      line = line.strip()
      if line.startswith(">"):
        name = line[1:]
        sequences[name] = ""
      else:
        sequences[name] += line
  return sequences

if you grab the gzip [compressed version of the Bc01 sequence](https://raw.githubusercontent.com/linsalrob/ComputationalGenomicsManual/master/Python/Bc01.fasta.gz), you can test this out.

In [ ]:
sequences = read_fasta("Bc01.fasta.gz")
for name, sequence in sequences.items():
  print(f"{name}: {len(sequence)}")

[Return to the lesson listing](#lessons)

[Return to the top of the notebook](#top)